# Transportation Model under changing commodity status

In [5]:
from pulp import *

# Sample data (modify according to your specific problem)
supply_nodes = ['Factory1', 'Factory2']
demand_nodes = ['Customer1', 'Customer2', 'Customer3']
commodities = ['Commodity1', 'Commodity2']

# Initial transportation costs
initial_transportation_costs = {
    ('Factory1', 'Customer1'): 10,
    ('Factory1', 'Customer2'): 15,
    ('Factory1', 'Customer3'): 20,
    ('Factory2', 'Customer1'): 12,
    ('Factory2', 'Customer2'): 18,
    ('Factory2', 'Customer3'): 22,
}

# Changing transportation costs due to commodity status
changing_transportation_costs = {
    ('Factory1', 'Customer1'): 12,
    ('Factory1', 'Customer2'): 20,
    ('Factory1', 'Customer3'): 25,
    ('Factory2', 'Customer1'): 15,
    ('Factory2', 'Customer2'): 22,
    ('Factory2', 'Customer3'): 28,
}

# Create a Linear Programming problem
transport_model = LpProblem(name="Transportation_Model", sense=LpMinimize)

# Decision variables
x = LpVariable.dicts("Shipment", ((i, j) for i in supply_nodes for j in demand_nodes), lowBound=0, cat='Integer')

# Objective function: minimize transportation costs
transport_model += lpSum(initial_transportation_costs[(i, j)] * x[(i, j)] for i in supply_nodes for j in demand_nodes)

# Constraints: supply and demand balance
for i in supply_nodes:
    transport_model += lpSum(x[(i, j)] for j in demand_nodes) <= 100  # Total supply capacity

for j in demand_nodes:
    transport_model += lpSum(x[(i, j)] for i in supply_nodes) >= 50  # Total demand requirement

# Solve the problem with initial transportation costs
transport_model.solve()

# Print the results with initial transportation costs
print("Optimal transportation plan (initial costs):")
for i in supply_nodes:
    for j in demand_nodes:
        if x[(i, j)].varValue > 0:
            print(f"Ship {x[(i, j)].varValue} units from {i} to {j}")

print(value(transport_model.objective))

# Update the transportation costs to changing commodity status
for (i, j), cost in changing_transportation_costs.items():
    initial_transportation_costs[(i, j)] = cost

# Update the objective function with changing transportation costs
transport_model.objective = lpSum(initial_transportation_costs[(i, j)] * x[(i, j)] for i in supply_nodes for j in demand_nodes)

# Solve the problem with changing transportation costs
transport_model.solve()

# Print the results with changing transportation costs
print("\nOptimal transportation plan (changing costs due to commodity status):")
for i in supply_nodes:
    for j in demand_nodes:
        if x[(i, j)].varValue > 0:
            print(f"Ship {x[(i, j)].varValue} units from {i} to {j}")

print(value(transport_model.objective))



Optimal transportation plan (initial costs):
Ship 50.0 units from Factory1 to Customer2
Ship 50.0 units from Factory1 to Customer3
Ship 50.0 units from Factory2 to Customer1
2350.0

Optimal transportation plan (changing costs due to commodity status):
Ship 50.0 units from Factory1 to Customer1
Ship 50.0 units from Factory1 to Customer3
Ship 50.0 units from Factory2 to Customer2
2950.0
